# Intro To Stream Ciphers

The purpose of this notebook is to show how lazy implementation of cryptographic algoritms, specifically the one-time pad, can
lead to easily breakable ciphers. In this case, I show how using anything but TRNG or CSRNG for the one-time pad can make an unbreakable cipher easy to break. Credit to *Understanding Cryptography* by Paar and Pelzl.

Stream ciphers encrypt bits individually, as opposed to block ciphers, which encrypt blocks at a time

**Encryption:** $y_i = e(x_i) = x_i + s_i \mod 2$

**Decryption:** $x_i = d(y_i) = y_i + s_i \mod 2$

Where $x_i, y_i, s_i$ consist of the plaintext, ciphertext, and key bit, respectively. In our case, we will be encrypting characters at a time.
Note that encryption and decryption are the same function, and that modulo 2 arithmetic is equivalent to the XOR function as:

$0 + 1 \mod 2 = 1,  0 \oplus 1 = 1$
$1 + 1 \mod 2 = 0,  0 \oplus 0 = 0$

The XOR = $\oplus$ function is used because, depending on the key bit $s_i$, there is a 50% chance for the the ciphertext $y_i$
is either a 0 or 1. Also, the same keystream can be used for decryption and encryption, which is not true for the OR, AND, or NAND gates. Specifically, the AND and NAND gates are not invertible.

The security of stream ciphers depends upon the unpreditability of the individual key bits $s_i$, and the random number generators used to generate them.
 

In [1]:
#import numpy to solve key bit system of equations
import numpy
#import random to show how it's use with the One-Time pad can be insecure
import random

#define our starting sample text
sample_text = "The red fox jumped over the log and dashed towards the river, away from the hunter"

#ecnryption and decryption for a single character
def encrypt_decrypt_char(c: chr, key: int) -> chr:
    return chr( (ord(c) ^ key) % 128 )
        
c = "a"
print("plain character: " + c)
result = encrypt_decrypt_char(c, 4)
#encrypted result
print("encrypted character: " + result)
result = encrypt_decrypt_char(result, 4)
#decrypted result
print("decrypted character: " + result)
        

plain character: a
encrypted character: e
decrypted character: a


# TRNG, PSNG, CSPRNG

True random number generators (TRNGs) cannot be reproduced. Pseudo Random Number Generators generate sequences which are derived from an initial seed value:

$s_{i+1} = f(s_i)$

cryptologically secure pseudo random number generators generate sequences in which if you know the bits $s_0, ... s_i$ it is possible, but computationally infeasable to generate the bits after $s_i$. A good CSPRNG produces bits indistinguishable from random bits.

# The One-Time Pad

The One-Time Pad is a stream cipher for which
1. the key stream $s_0, s_1, ..., s_i$ is generated by a true random number generator
2. The key stream is only know to the legitimate communicating parties
3. every key stream bit $s_i$ is only used once

The One-Time Pad is unconditionally secure (cannot be broken, even with infinite resources) as each indivual relation is a linear equation with 2 unknowns. Even if the attacker knows the ciphertext, the key bit is equally likely to be 0 or 1, thus the plaintext is equally likely to be 0 or 1. Even if the attacker correctly guesses the plain bit $x_i$ or key bit $s_i$ there is no way to know he guessed correctly, or to derive the next key bit $s_{i+1}$ from his correct guess.

Note that in the implementation below, making the key a constant number (i.e 5), makes the one time pad susceptible to letter frequency analysis vs when using the python random class. Also, note that the seed must be known for decryption as the Python random class is a PRNG, we need the initial value to construct the sequence. Removing the second seed results in garbage decrypted text.

In [2]:
#implementation of one-time pad with python random() as the key stream (so, not a true One-Time Pad)
#encrypts plain text and decrypts cipher text

random.seed(5)
key_stream = list()
def one_time_pad(text: str) -> str:
    result = ""
    for i in text:
        key = random.randint(0, 10)
        key_stream.append(key)
        result += encrypt_decrypt_char(i, key)
    return result

#result of encryption
enc_text = one_time_pad(sample_text)
print("encrypted text: " + enc_text)
#result of decryption
random.seed(5)
dec_text = one_time_pad(enc_text)
print("decrypted text: " + dec_text)

encrypted text: ]l`*zec#loz!ornvme)lvft$vng!nfn'cld gbqjaa#|e}bpgu$tmc"pmw`v%)a~k|!bwkj%vob"hqnqcr
decrypted text: The red fox jumped over the log and dashed towards the river, away from the hunter


# Issues With the One-Time Pad

While the One-Time pad is unconditionally secure (cannot be broken even with infinite computational resources), it isn't widely used for a variety of reasons. Since every key bit is unique and cannot be reused, the key is as along as the message. So, encrypting a 1 MB file means also sharing a 1 MB key. Issues also arise in keeping the key stream only know to legitimate communication parties. How would this key be shared? For example: If a One-Time pad is used to encrypt an email message, and the key stream is also sent via email, the key stream itself would need to be encrypted. This presents obvious security vulnerabilities. Even if the sender and receiver of the email agree on a truly random key stream before hand, they would have to agree on another key stream to send another email. You can see how this can become impractical.

# Issues With This Implementation

This implementation of the One-Time Pad encrypts characters at a time, not bits. This reduces the solution set for the key stream. For example, the 16 bits = 2 bytes:

$10101010 10101010$

has $16!$ possiblities for key stream if encrypted bit by bit, while if we encrypt them as bytes, we have $2!$ possibilities for the key stream. Having a printable character also makes it easier to guess.

# Weaknesses of Key Streams From PRNGs

Assume that an attacker knows we have generated our key stream from a PRNG based on linear congruential generator (as many PRNG are). So, the attacker knows that the key bit used to encrypt byte $S_{i+1}$ is $S_{i+1} = f(S_i)$. If the attacker has some information to any key bit $S_i$, all key bits after $s_i$ can be expressed as:

$S_{i+1} = A * S_i + B \mod m$

$S_{i+2} = A * S_{i+1} + B \mod m$

Assume an attacker is trying to decipher our sample text above, and he knows the first byte based off of file header information, or even an educated guess. Since $A, B$ are constant for each key bit, an attacker would only need to know some decrypted text and he would be able to dervive $A, B,$ and every $S_i$ by constructing the system of equations:

$A = (S_{i+1} - S_{i+2})/(S_{i+1} - S_{i+1}) \mod m$

$B = S_3 - S_1(S_2 - S_3)/(S_1 - S_2) \mod m$


In [3]:
#assume the attacker knows the first byte 
#assume the attacker knows we are using a stream cipher, and the modulus is 2
def derive_k_0(cipher_text: str, plain_text: str, k_0: int) -> list():
    s = list()
    l_plain, l_cipher = len(plain_text), len(cipher_text)
    if(l_plain != l_cipher):
        print("Error, cipher text and plain text must be same length")
        return None
    for i in range(l_plain):
        y_i, x_i = cipher_text[i], plain_text[i]
        s_i = ord(y_i) + (ord(x_i) % 2)
        s.append(s_i)
    return s

sub_enc, sub_dec = enc_text[0: 4], dec_text[0:4]
k_0 = derive_k_0(sub_enc, sub_dec, 5)
print(k_0)

[93, 108, 97, 42]


Assume $S_i$ represents the first $i$ bits of the key stream held in the variable s

In [4]:
#derive S_i from the key stream, and construct A, B from S_i
def find_A_B(s: list):
    if(len(s) < 4):
        print("not enough information")
        return None
    A = (s[1] - s[2])/(s[0] - s[1]) % 2
    B = s[1] - s[0] * (s[1] - s[3])/(s[0] - s[1]) % 2
    return A, B
A, B = find_A_B(k_0)
print("A = " + str(A))
print("B = " + str(B))

A = 1.2666666666666666
B = 107.19999999999999


Now that we have derived $A, B,$ we can use them to find the rest of the key stream through the relationship:
$S_{i+1} = A * S_i + B \mod 2$

In [5]:
#A, B = constants used to derive the rest of the keys, l = length of text (for how many keys to generate)
#k_0 = initial keys derive
def derive_k(A: float, B: float, l: int, k_0: list):
    k = k_0
    start, stop = 4, l
    for i in range(start, stop):
        s_1 = A * k[i-1] + B % 2
        k.append(s_1)
    return k

l = len(sample_text)
k = derive_k(A, B, l, k_0)
print(k)


[93, 108, 97, 42, 54.399999999999984, 70.10666666666663, 90.00177777777772, 115.20225185185176, 147.12285234567887, 187.5556129711932, 238.7704430968447, 303.64256125600326, 385.8139109242708, 489.89762050407626, 621.7369859718299, 788.7335155643177, 1000.2624530481357, 1268.1991071943053, 1607.5855357794533, 2037.4750119873074, 2582.001681850589, 3271.7354636774126, 4145.398253991389, 5252.0377883890915, 6653.781198626182, 8429.322851593164, 10678.342278684675, 13527.100219667254, 17135.526944911857, 21706.20079688835, 27495.721009391913, 34829.11327856308, 44118.07681951323, 55884.09730471676, 70787.72325264123, 89665.64945334554, 113577.68930757101, 143866.27312292327, 182231.81262236947, 230828.16265500133, 292383.5393630017, 370353.6831931355, 469115.8653779716, 594214.629478764, 752673.0640064343, 953387.0810748166, 1207624.836028101, 1529659.3256355943, 1937569.6791384192, 2454256.1269086646, 3108725.6274176417, 3937720.328062346, 4987780.282212305, 6317856.224135586, 8002619.08

Since the rand() function in python is based on the Mersenne Twister and not a Linear Congruential Generator, predicting it's output involves a more complex calculation - though still possible. In the interest of keeping this notebook on topic, I will omit this portion.